In [1]:
import pandas as pd
from textblob import TextBlob
pd.set_option('display.max_colwidth', 20000)

In [2]:
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

In [18]:
data=pd.read_csv('asset/model_use/origin_data_01.csv')

In [19]:
from bs4 import BeautifulSoup
import re
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
   
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [20]:
data_clean = []
for t in list(data['text']):
    data_clean.append(tweet_cleaner(t))

In [21]:
data['tidy_text']=data_clean

In [22]:
list_polarity=[]
list_subjectivity=[]
for text in data_clean:
    blob=TextBlob(text)
    list_polarity.append(blob.sentiment.polarity)
    list_subjectivity.append(blob.sentiment.subjectivity)

In [23]:
data['polarity']=list_polarity
data['subjectivity']=list_subjectivity

In [24]:
df=data.assign(tone = lambda dataframe: 
          dataframe['polarity'].map(lambda polarity: 
                                    'positive' if polarity > 0 
                                    else( 'neutral' if polarity ==0 else 'negative')))

In [25]:
df=df.set_index('tweet_id')

### after manual labeling adjustment

In [26]:
# df.to_csv('asset/model_use/fresh_data.csv')